In [1]:
%pylab inline 

Populating the interactive namespace from numpy and matplotlib


In [2]:
cd ../TileEM/

/home/jlee782/crowd-seg/analysis/TileEM


In [ ]:
# Run the PR calculation for PixelEM on various thresholds based on  the adjacency and the non-adjacency ML construction
# Creates a Pixel_PR.csv in each of the base_dir/dir_name directory
import pandas as pd
import pickle as pkl
from qualityBaseline import *
from calc_Tstar import *
import os
import time
from PixelEM import *
base_dir= "pixel_em"
start = time.time()
worker_Nbatches={5:10,10:8,15:6,20:4,25:2,30:1}
sampleN_lst=worker_Nbatches.keys()
from TileEM_plot_toolbox import *
start = time.time()
PR_pixelEM = pd.read_csv("pixel_em/full_PRJ_table.csv")
PR_pixelEM_GT = pd.read_csv("pixel_em/GTfull_PRJ_table.csv")
#PR_pixelEM_GTLSA = pd.read_csv("pixel_em/GTLSAfull_PRJ_table.csv")
df = pd.read_csv("../computed_my_COCO_BBvals.csv",index_col=0)
tbl=[]
col_lst=[]
i=0
cols = [u'Num Points',u'Area Ratio',u'Jaccard [Self]', u'Precision [Self]', u'Recall [Self]']
for Nworker in sampleN_lst:
    for batch_num in range(worker_Nbatches[Nworker]):
        dir_name = "{0}workers_rand{1}".format(Nworker,batch_num)
        print "Working on :", dir_name
        #os.chdir(base_dir+"/"+dir_name)
        tmp_tbl=[]
        #for i,fname in enumerate(glob("obj*")):
        #    objid=int(fname[3:])
        object_lst = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47]
        for objid in tqdm(object_lst):
            tmp_tbl=[objid,Nworker,batch_num]
            if i==0: col_lst = ["object_id","Nworker","batch_num"]
            #Summarization Based metrics
            workers = ast.literal_eval([line for line in open("pixel_em/"+dir_name+"/obj{}/worker_ids.json".format(objid))][0])
            filtered_df = df[(df["worker_id"].isin(workers))&(df["object_id"]==objid)] #only look at summarization scores of sampled workers
            if len(filtered_df)!=0:
                for attr in cols:
                    if i==0:
                        col_lst.extend(["P [{}]".format(attr),"R [{}]".format(attr),"J [{}]".format(attr)])
                    best_worker_BB = filtered_df[filtered_df[attr]==filtered_df[attr].max()]
                    best_worker_id = int(best_worker_BB["worker_id"].sample(1).values)
                    best_worker_mask = pkl.load(open("pixel_em/obj{}/mask{}.pkl".format(objid,best_worker_id)))
                    gt_mask = pkl.load(open("pixel_em/obj{}/gt.pkl".format(objid)))
                    p,r,j = get_precision_recall_jaccard(best_worker_mask, gt_mask)
                    tmp_tbl.extend([p,r,j])
        tbl.append(tmp_tbl)
        i+=1

/home/jlee782/.local/lib/python2.7/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
  0%|          | 0/44 [00:00<?, ?it/s]

Working on : 5workers_rand0


  0%|          | 0/44 [00:00<?, ?it/s]         

Working on : 5workers_rand1


  0%|          | 0/44 [00:00<?, ?it/s]         

...

  0%|          | 0/44 [00:00<?, ?it/s]         

Working on : 5workers_rand5


  0%|          | 0/44 [00:00<?, ?it/s]         

Working on : 5workers_rand6


  2%|▏         | 1/44 [00:06<04:40,  0.15it/s]

In [ ]:
df = pd.DataFrame(tbl,columns=col_lst)
df.to_csv("pixel_em/summarization_PRJ_table.csv")
end = time.time()
print "Time Elapsed: ", end-start

In [ ]:
PR_pixelEM = pd.read_csv("pixel_em/full_PRJ_table.csv")
PR_pixelEM_GT = pd.read_csv("pixel_em/GTfull_PRJ_table.csv")


In [ ]:
PR_pi